import libraries

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from utils import load_json, load_pickle

: 

Settings

In [3]:
baseline_folder = "../FP_prediction/baseline_models/models_cached/"
MIST_folder = "../FP_prediction/mist/models_cached/"

Helper Functions

In [2]:
def load_json(path):

    with open(path, "r") as f:

        return json.load(f)

Consolidate results

In [8]:
# Manually add in (hack)

all_folders = []
all_test_performances = {"loss": {}, "jaccard": {}} 

for folder in [MIST_folder, baseline_folder]:

    for model_type in os.listdir(folder):
        subfolder = os.path.join(folder, model_type)
        for dataset in os.listdir(subfolder):
            subsubfolder = os.path.join(subfolder, dataset)
            for checkpoint in os.listdir(subsubfolder):
                all_folders.append(os.path.join(subsubfolder, checkpoint))

for f in all_folders:

    try:
        expt_name = f.split("/")[-1]
        test_performance = load_json(os.path.join(f, "test_performance.json"))
        for k, v in test_performance.items():
            all_test_performances[k][expt_name] = round(v, 5)
    except Exception as e:
        print(e)
        continue

Format the table

In [9]:
results = pd.DataFrame.from_dict(all_test_performances)
results = results.reset_index()
results = results.rename(columns = {"index": "expt_name"})
results.loc[:, "dataset"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[0]) 
results.loc[:, "meta"] = results.loc[:, "expt_name"].apply(lambda x: "meta" in x) 
results.loc[:, "model"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[1]) 
results.loc[:, "split"] = results.loc[:, "expt_name"].apply(lambda x: "_".join(x.split("_")[-2:]).replace("4096_", "")) 

results = results.loc[:, ["dataset", "model", "split", "meta", "loss", "jaccard"]]

results = results.sort_values(by = 'split', ascending = False)
results = results.sort_values(by = 'model', ascending = False)
results = results.sort_values(by = 'dataset', ascending = False)

Seperate results into w and wo meta

In [10]:
results_wo_meta = results.loc[results.loc[:, "meta"].apply(lambda x: x is False), :]
results_w_meta = results.loc[results.loc[:, "meta"].apply(lambda x: x is True), :]

In [11]:
for dataset in ["NIST2023", "MSG", "C"]:
    for split in ["random", "scaffold_vanilla", "inchikey_vanilla"]:

        sub_results_wo_meta = results_wo_meta.loc[results_wo_meta.apply(lambda x : x["dataset"] == dataset and x["split"] == split, axis = 1), :]
        print(sub_results_wo_meta)
        print("*" * 80)
        print() 

     dataset model   split   meta     loss  jaccard
10  NIST2023  MIST  random  False  0.01236  0.69384
1   NIST2023  MIST  random  False  0.01628  0.62105
********************************************************************************

     dataset model             split   meta     loss  jaccard
0   NIST2023  MIST  scaffold_vanilla  False  0.06124  0.25526
11  NIST2023  MIST  scaffold_vanilla  False  0.06166  0.25275
********************************************************************************

    dataset model             split   meta     loss  jaccard
9  NIST2023  MIST  inchikey_vanilla  False  0.04723  0.30194
2  NIST2023  MIST  inchikey_vanilla  False  0.04789  0.30135
********************************************************************************

   dataset    model   split   meta     loss  jaccard
7      MSG     MIST  random  False  0.02223  0.67438
39     MSG       MS  random  False  0.04291  0.36947
38     MSG  formula  random  False  0.03366  0.48250
*****************

In [13]:
for dataset in ["NIST2023", "MSG", "C"]:
    for split in ["random", "scaffold_vanilla", "inchikey_vanilla"]:

        sub_results_w_meta = results_w_meta.loc[results_w_meta.apply(lambda x : x["dataset"] == dataset and x["split"] == split, axis = 1), :]
        print(sub_results_w_meta)
        print("*" * 80)
        print() 

     dataset model   split  meta     loss  jaccard
19  NIST2023  MIST  random  True  0.01628  0.62107
********************************************************************************

     dataset model             split  meta     loss  jaccard
17  NIST2023  MIST  scaffold_vanilla  True  0.05966  0.25595
********************************************************************************

     dataset model             split  meta     loss  jaccard
18  NIST2023  MIST  inchikey_vanilla  True  0.04787  0.30027
********************************************************************************

   dataset    model   split  meta     loss  jaccard
29     MSG  formula  random  True  0.03435  0.47187
12     MSG     MIST  random  True  0.02274  0.66692
********************************************************************************

   dataset model             split  meta     loss  jaccard
11     MSG  MIST  scaffold_vanilla  True  0.06252    0.273
****************************************************